In [1]:
import pandas as pd
import numpy as np

In [2]:
import time
from datetime import date

In [3]:
data = pd.read_csv('marketing_customer_analysis.csv')

In [4]:
data.shape

(10910, 26)

In [5]:
data.rename(columns={'EmploymentStatus':'Employment Status'}, inplace=True)
data.rename(columns={'Unnamed:' ' 0':'unnamed:0'}, inplace=True)

columns = list(map(lambda x: x.strip('').lower().replace(' ','_'), data.columns))

#My first code with for loop
#columns=[]
#for i in range(len(data.columns)):
#    columns.append(data.columns[i].strip('').lower().replace(' ','_'))

#data.columns.lowercase()

data.columns = columns
data.tail()

,unnamed:0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN
10908,10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A
10909,10909,IV32877,NaN,6857.519928,NaN,Basic,Bachelor,1/8/11,Unemployed,M,...,0.0,3,Personal Auto,Personal L1,Offer4,Web,1021.719397,SUV,Medsize,NaN


In [6]:
data_numerical= data._get_numeric_data()
data_numerical.columns

Index(['unnamed:0', 'customer_lifetime_value', 'income',
       'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'total_claim_amount'],
      dtype='object')

In [7]:
data_categorical = [column for column in data.columns if data[column].dtype=="O"]
data_categorical

['customer',
 'state',
 'response',
 'coverage',
 'education',
 'effective_to_date',
 'employment_status',
 'gender',
 'location_code',
 'marital_status',
 'policy_type',
 'policy',
 'renew_offer_type',
 'sales_channel',
 'vehicle_class',
 'vehicle_size',
 'vehicle_type']

In [8]:
nullsdata= pd.DataFrame(round(data.isna().sum()/len(data),4)*100)
nullsdata = nullsdata.rename(columns={'index':'header',0:'proportion_of_nulls'})
nullsdata

,proportion_of_nulls
unnamed:0,0.00
customer,0.00
state,5.78
customer_lifetime_value,0.00
response,5.78
coverage,0.00
education,0.00
effective_to_date,0.00
employment_status,0.00
gender,0.00


In [9]:
data.isna().all(axis=1).value_counts() # no full Null row.

False    10910
dtype: int64

In [10]:
data.drop(['vehicle_type'], axis = 1, inplace = True)  # There are a lot of null entries, so it makes sense just to remove it.

In [11]:
data['state']=data['state'].fillna('Unknown')

In [12]:
data['response']=data['response'].fillna('Unknown')

In [13]:
data['vehicle_class']=data['vehicle_class'].fillna('Unknown')

In [14]:
data['vehicle_size']=data['vehicle_size'].fillna('Unknown')

In [15]:
#data[(data['data['number_of_open_complaints'].isna()].isna() & data['months_since_last_claim'].isna()) == True]
# number of open complaints and months since last claim rows are Null at the same time.

comp_mean = data['number_of_open_complaints'].mean()
data['number_of_open_complaints']=data['number_of_open_complaints'].fillna(comp_mean)

month_mean = data['months_since_last_claim'].mean()
data['months_since_last_claim']=data['months_since_last_claim'].fillna(month_mean)

data.head()

,unnamed:0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,52,0.000000,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,26,0.000000,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,31,0.000000,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,3,0.000000,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,31,0.384256,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize


In [16]:
data['effective_to_date'] = pd.to_datetime(data['effective_to_date'],errors = 'coerce')

In [17]:
data['month'] = pd.DatetimeIndex(data['effective_to_date']).month

In [18]:
data['month'] = data['effective_to_date'].dt.strftime('%b')
data.head()

,unnamed:0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,0.000000,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,Feb
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,0.000000,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,Jan
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,0.000000,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,Feb
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,0.000000,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,Jan
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,0.384256,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,Jan


In [19]:
filtered = data.query("month == ['Jan', 'Feb', 'March']")

if filtered.shape == data.shape:  
    print('This data is just for the first quarter.')

This data is just for the first quarter.


In [21]:
# if it is necessary we can find means of all categorical values.
#means =[]
#for i in data_numerical.columns:
#    means.append(data[i].mean()

#or by using map

means = list(map(lambda x: data[x].mean(), data_numerical.columns))
print(means)

[5454.5, 8018.241093723163, 37536.28478460128, 93.19605866177818, 15.149070740488618, 48.09193400549954, 0.3842561058674663, 2.979193400549954, 434.88832970283863]
